In [1]:
# Mount Google Driver
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
drive.mount(ROOT)           # we mount the google drive at /content/drive
# change to clrs directionary
%cd "/content/drive/My Drive/Colab Notebooks/fluent_python_notes"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/fluent_python_notes


In [0]:
%mkdir ch12
!touch ch12/__init__.py

In [0]:
import imp

## 12.1 子类化内置类型很麻烦

- Python2.2 之前，内置类型（如 `list` 和 `dict`） 不能之类化
- Python2.2 之后，内置类型可以子类化
  - 但内置类型（使用 C 语言编写）不会调用用户定义的类覆盖特殊方法， 参见 [Subclasses of built-in types](https://pypy.readthedocs.io/en/latest/cpython_differences.html#subclasses-of-built-in-types)
  > - 至于内置类型的子类覆盖的方法会不会隐式调用，CPython 没有制定官方规则  
   - 基本上，内置类型的方法不会调用子类覆盖的方法
    - 例如，dict 的子类覆盖的 `__getitem__()` 方法不会被内置类型的 `get()` 方法调用
- 原生类型的这种行为违背了面向对象编程的一个基本原则
  - 始终应该从实例（`self`）所属的类开始搜索方法，即使在超类实现的类中调用也是如此
  - `__missing__` 方法（参见 3.4.2 节）能按预期方式工作，不过这只是特例
- 不只实例内部的调用有这个问题，内置类型的方法调用的其他类的方法，如果被覆盖了，也不会被调用
***
- 直接子类化内置类型（如 dict、list 或 str）容易出错，因为内置类型由 C 语言实现， 其通常会忽略用户覆盖的方法。
- 用户自己定义的类应该继承 [collections 模块](http://docs.python.org/3/library/collections.html) 中的类，例如 `UserDict`、`UserList` 和 `UserString`，这些类做了特殊设计，因此易于扩展
  - 这些类是使用 Python 编写的类，不会受到上述影响

###### 示例 12-1　内置类型 `dict` 的 `__init__` 和 `__update__` 方法会忽略我们覆盖的 `__setitem__` 方法

In [0]:
class DoppelDict(dict):
  def __setitem__(self, key, value):
    super().__setitem__(key, [value] * 2)

In [0]:
dd = DoppelDict(one=1)  # 继承自 dict 的 __init__ 方法忽略了覆盖的 __setitem__方法：'one' 的值没有重复
dd

{'one': 1}

In [0]:
dd['two'] = 2  # [] 运算符会调用我们覆盖的 __setitem__ 方法，按预期那样工作：'two' 对应的是两个重复的值，即 [2, 2]
dd

{'one': 1, 'two': [2, 2]}

In [0]:
dd.update(three=3)  # 继承自 dict 的 update 方法也不使用我们覆盖的 __setitem__ 方法：'three' 的值没有重复
dd

{'one': 1, 'three': 3, 'two': [2, 2]}

###### 示例 12-2　`dict.update` 方法会忽略 `AnswerDict.__getitem__` 方法

In [0]:
class AnswerDict(dict):
  def __getitem__(self, key):
    return 42

In [0]:
ad = AnswerDict(a='foo')
ad['a']

42

In [0]:
d = {}
d.update(ad)  # d 是 dict 的实例，使用 ad 中的值更新 d
d['a']  # dict.update 方法忽略了 AnswerDict.__getitem__ 方法

'foo'

In [0]:
d

{'a': 'foo'}

###### 示例 12-3　`DoppelDict2` 和 `AnswerDict2` 能像预期那样使用，因为它们扩展的是 `UserDict`，而不是 `dict`

In [0]:
import collections
class DoppelDict2(collections.UserDict):
  def __setitem__(self, key, value):
    super().__setitem__(key, [value] * 2)

In [0]:
dd = DoppelDict2(one=1)
dd

{'one': [1, 1]}

In [0]:
dd['two'] = 2
dd

{'one': [1, 1], 'two': [2, 2]}

In [0]:
dd.update(three=3)
dd

{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}

***

In [0]:
class AnswerDict2(collections.UserDict):
  def __getitem__(self, key):
    return 42

In [0]:
ad = AnswerDict2(a='foo')
ad['a']

42

## 12.2 多重继承和方法的解析顺序

- 任何实现多重继承的语言都要处理潜在的命名冲突，这种冲突由不相关的祖先类实现同名方法引起。这种冲突称为“菱形问题”
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200516145400.png width=700>

###### 示例 12-4　`diamond.py`：图 12-1 中的 A、B、C 和 D 四个类

In [0]:
%%writefile ch12/diamond.py
class A:
  def ping(self):
    print('ping:', self)

class B(A):
  def pong(self):
    print('pong:', self)

class C(A):
  def pong(self):
    print('PONG', self)

class D(B, C):
  def ping(self):
    super().ping()
    print('post-ping:', self)

  def pingpong(self):
    self.ping()
    super().ping()
    self.pong()
    super().pong()
    C.pong(self)

Overwriting ch12/diamond.py


###### 示例 12-4　diamond.py：图 12-1 中的 A、B、C 和 D 四个类

In [0]:
from ch12.diamond import *
d = D()
d.pong()  # 直接调用 d.pong() 运行的是 B 类中的版本

pong: <ch12.diamond.D object at 0x7fe3d62d0c50>


In [0]:
C.pong(d)  # 超类中的方法都可以直接调用，此时要把实例作为显式参数传入

PONG <ch12.diamond.D object at 0x7fe3d62d0c50>


###### 方法解析顺序（Metho Resolution Order, MRO）

- Python 能区分 `d.pong()` 调用的是哪个方法，是因为 Python 会按照方法解析顺序（Method ResolutionOrder，MRO）遍历继承图
- 类都有一个名为 `__mro__` 的属性，它的值是一个元组，按照方法解析顺序列出各个超类，从当前类一直向上，直到object 类

In [0]:
D.__mro__

(ch12.diamond.D, ch12.diamond.B, ch12.diamond.C, ch12.diamond.A, object)

- 若想把方法调用委托给超类，推荐的方式是使用内置的 `super()` 函数
  - 使用 `super()` 最安全，也不易过时
  - 调用框架或不受自己控制的类层次结构中的方法时，尤其适合使用 `super()`
  - 使用 `super()` 调用方法时，会遵守方法解析顺序
- 如果需要绕过方法解析顺序，可以直接调用某个超类的方法
  - ```python
    def ping(self):
      A.ping(self)  # 必须显示传入 self 参数，因为这样访问提未绑定方法（unbound method）
      print('post-ping', self)
  ```
- 方法解析使用 C3 算法计算，可参考 [Python 2.3 Method Resolution Order](https://www.python.org/download/releases/2.3/mro/)

###### 示例 12-6　使用 `super()` 函数调用 `ping` 方法

In [0]:
from ch12.diamond import D
d = D()  # super 函数把 ping 调用委托给 A类；这一行由 A.ping 输出
d.ping()

ping: <ch12.diamond.D object at 0x7fe3d62f9748>
post-ping: <ch12.diamond.D object at 0x7fe3d62f9748>


###### 示例 12-7　pingpong 方法的 5 个调用

In [0]:
import ch12.diamond
imp.reload(ch12.diamond)
from ch12.diamond import D
d = D()
d.pingpong()

ping: <ch12.diamond.D object at 0x7fe3d62d0320>
post-ping: <ch12.diamond.D object at 0x7fe3d62d0320>
ping: <ch12.diamond.D object at 0x7fe3d62d0320>
pong: <ch12.diamond.D object at 0x7fe3d62d0320>
pong: <ch12.diamond.D object at 0x7fe3d62d0320>
PONG <ch12.diamond.D object at 0x7fe3d62d0320>


- 第一个调用是 `self.ping()`，运行的是 D 类的 `ping` 方法，输出这一行和下一行
- 第二个调用是 `super().ping()`，跳过 D 类的 `ping` 方法，找到 A 类的 `ping` 方法
- 第三个调用是 `self.pong()`，根据 `__mro__`，找到的是 B 类实现的 `pong` 方法
- 第四个调用是 `super().pong()`，也根据 `__mro__` ，找到 B 类实现的 `pong` 方法
- 第五个调用是 `C.pong(self)`，忽略 `mro` ，找到的是 C 类实现的 `pong` 方法

###### 示例 12-8　查看几个类的 `__mro__` 属性

In [0]:
bool.__mro__  # bool 从 int 和 object 中继承方法和属性

(bool, int, object)

In [3]:
def print_mro(cls):
  print(', '.join(c.__name__ for c in cls.__mro__))
print_mro(bool)

bool, int, object


In [0]:
from ch11.frenchdeck2 import FrenchDeck2
print_mro(FrenchDeck2)  # FrenchDeck2 类的祖先包含 collections.abc 模块中的几个抽象基类

FrenchDeck2, MutableSequence, Sequence, Reversible, Collection, Sized, Iterable, Container, object


In [0]:
import numbers
print_mro(numbers.Integral)

Integral, Rational, Real, Complex, Number, object


In [0]:
import io
print_mro(io.BytesIO)

BytesIO, _BufferedIOBase, _IOBase, object


In [0]:
print_mro(io.TextIOWrapper)

TextIOWrapper, _TextIOBase, _IOBase, object


###### tkinter 模块

In [0]:
import tkinter
print_mro(tkinter.Text)

Text, Widget, BaseWidget, Misc, Pack, Place, Grid, XView, YView, object


- <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200516155554.png width=800>

## 12.3 多重继续的真实应用

- [Tkinter](https://docs.python.org/3/library/tkinter.html) 模块把多重继承用到了极致，其 GUI 类层次结构的 UML 结构图如下
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200520065453.png width=800>
  - 几个类的含义
    1. Toplevel：表示 Tkinter 应用程序中顶层窗口的类。
    2. Widget：窗口中所有可见对象的超类。
    3. Button：普通的按钮小组件。
    4. Entry：单行可编辑文本字段。
    5. Text：多行可编辑文本字段
  - 解析顺序如下：

In [4]:
import tkinter
print_mro(tkinter.Toplevel)

Toplevel, BaseWidget, Misc, Wm, object


In [5]:
print_mro(tkinter.Widget)

Widget, BaseWidget, Misc, Pack, Place, Grid, object


In [6]:
print_mro(tkinter.Button)

Button, Widget, BaseWidget, Misc, Pack, Place, Grid, object


In [7]:
print_mro(tkinter.Entry)

Entry, Widget, BaseWidget, Misc, Pack, Place, Grid, XView, object


In [8]:
print_mro(tkinter.Text)

Text, Widget, BaseWidget, Misc, Pack, Place, Grid, XView, YView, object


## 12.4 处理多重继承

- 使用多重继随容易得出令人费解和脆弱的设计，以下是避免把类图搅乱的一些建议
  1. 把接口继承和实现继承区分开
    - 继随接口，创建子类型，实现“是什么”关系
    - 继承实现，通过重用避免代码重复
  2. 使用抽象基类定义接口
    - 现代的 Python 中，如果类的作用是定义接口，应该明确把它定义为抽象基类
  3. 通过混入重用代码
    - 如果一个类的作用是为多个不相关的子类提供方法实现，从而实现重用，但不体现“是什么”关系应该把那个类明确地定义为混入类（mixin class）
  4. 在名称中明确指明混入
    - 因为在 Python 中没有把类声明为混入的正规方式，所以应在名称中加入 ...Mixin 后缀
  5. 抽象基类可以作为混入，反过来则不成立
    - 抽象基类可以实现具体方法，因此也可以作为混入使用
    - 抽象基类可以作为其他类的唯一基类，而混入决不能作为唯一的超类
  6. 不要子类化多个具体类
    - 具体类可以没有，或最多只有一个具体超类
    - 即具体类的超类中除了这一个具体超类之外，其余的都是抽象基类或混入
  7. 为用户提供聚合类
    - 如果抽象基类或混入的组合对客户代码非常有用，那就提供一个类，使用易于理解的方式把它们结合起来
    - 这种类即称为聚合类（aggregate class）
    - 示例：[tkinter.Widget 类的完整代码](https://hg.python.org/cpython/file/3.4/Lib/tkinter/__init__.py#l2141)
      - ```python
          class Widget(BaseWidget, Pack, Place, Grid):
            """Internal class.
            Base class for a widget which can be positioned with the
            geometry managers Pack, Place or Grid."""
            pass
      ```
  8. 优先使用对象组合，而不是类继承
    - 优先使用组合能让设计更灵活
    - 因为子类化是一种紧耦合，而且较高的继承树容易倒

### Tkinter 好的、不好的和令人厌恶的方面

- `Misc` 名称本身明显是代码异味，其本身有 100 多个方法，而所有小组件都继承它
  - 每个小组件不会都要处理剪切板、文本选择和计时器等
  - 我们可能不能把文本粘贴到按钮上，也不能选择滚动条里的文字
- `Misc` 应该拆分成几个专门的混入类，而且不是所有小组件都应该继承这些混入

In [10]:
len(dir(tkinter.Misc))

190

In [13]:
len(dir(tkinter.Button))  # 当查找某个需要的方法时，方法过多，极其不方便

210

## 12.5 一个现代示例： Django 通用视图中的混入

- 在 Django 中，视图是可调用的对象，它的参数是表示 HTTP 请求的对象，返回值是一个表示 HTTP 响应的对象
- Django 1.3 引入了基于类的视图，而且还通过基类、混入和拿来即用的具体类提供了一些通用视图类，如下图所示
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200520072702.png width=700>
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200520073416.png width=700>
  - `View` 是所有视图（可能是个抽象基类）的基类，提供核心功能，如 `dispatch` 方法
  - `TemplateResponseMixin` 提供的功能只针对需要使用模板的视图
  - `ListView` 是一个聚合类，不含任何代码（定义体中只有一个文档字符串）
  - `BaseListView` 类把 `View` 和 `MultipleObjectMixin` 的功能整合在一起，避免了模板机制的开销，能够方便的创建一个使用模板渲染的视图
- 与 `Tkinter` 相比，`Django` 基于类的视图 API 是多重继承更好的示例
  - Django 的混入类易于理解
  - 各个混入的目的明确，而且名称的后缀都是 `...Mixin`
- 基于类的视图能避免大量样板代码，便于重用，还能增进团队交流
  - 例如，为模板和传给模板上下文的变量定义标准的名称